# Deploy Media Graph

In this section, we will deploy media graphs onto our IoT Edge device to trigger our modules to perform certain actions.

## Deploy / Manage Media Graph on LVA Edge Module
At this stage of the sample, we should have all required modules deployed to the Edge device, up and running.  

Now we will deploy media graphs onto the "lvaEdge" module. Here, "lvaEdge" is the name that we assigned to the LVA module inside the module deployment template json file from the [previous sample](08_deploy_iotedge_modules.ipynb). Media graph is also a Json file that defines a media flow pipeline from media sources to the target sinks and any analytics processes in-between. The media graph file consists of following modules:  

```
{
    "@apiVersion": "1.0",

    "name": "SampleMediaGraphOrganization",

    "properties":{

        "parameters": ...

        "sources": ...

        "processes": ...

        "sinks": ...
    }
}
```

In the media graph Json file above, the "parameters" section is the most critical part of the media graph. The "parameters" section defines all required parameters to run the media graph. In this section, we will define the addresses of the media sources, their access credentials, and the parameters related to the process modules, such as fps rate and sensitivity of motion detection. Each media graph may have a different media flow, i.e., one may have a motion detection processor while another may not. You may modify this media graph according to your needs.  

### Sample Media Graph
In this sample project, we provide a sample media graph file [template.json](../../../../MediaGraph/topologies/motion-with-httpExtension/topology.json) under the MediaGraph/topologies/motion-with-httpExtension folder.  The media graph in this file ingests a video stream from a single IP camera, runs motion detector processor on the stream, and sends the stream to one of two targets if there is motion. The first target is Azure Media Services, a cloud platform that can archive the stream's motion chunks. The other possible target is the custom AI module that we developed previous sections of this sample; the AI module analyzes the stream (a chunk with motion) and then sends the result (a Json-format inference result) to IoT Hub.

### Setting Parameters in Media Graph
There are two ways to set parameters in the media graph:  
1. Update the default values of the parameters in "template.json", which is **not** the prefered method. 
2. Note the parameter names and data types in "template.json", and override their values in the deployment command that we will show in the next section.

### Deploy the Media Graph with Custom Parameters
Below, we will explain the following operations:
1. Setting (deploying) Media Graph Topology on the IoT Edge Device's lvaEdge module
2. Creating a Media Topolgy Instance from the one set in step 1
3. Activate the Topology Instance that set in step 2

We will also show how to
* List and delete the already set Graph Topologies on the Device
* List and delete the already set Graph Instances on the Device
* Acticate / deactivate the Graph Instances

## Install Prerequisites
We will use a simple Python script to deploy our media graph Json file into the "lvaEdge" module. Our process entails sending a file into the IoT Edge device over the Internet through a IoT Edge Hub service. To do so, we need to install following Python packages. These packages were developed by Microsoft and help with interacting with IoT Hub Services.

!Important: As mentioned in the first section of this tutorial, you must use the right ```pip``` command (```pip``` or ```pip3```) depending on your Python installation. This sample is tested with Python version 3.6 and so we use the ```pip3``` command to install the packages into our Python3 environment.

In [1]:
!pip install azure-iot-device
!pip install azure-iot-hub

     |████████████████████████████████| 138 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 3.1 MB/s  eta 0:00:01
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.0-py3-none-any.whl size=61415 sha256=8ada862a1071dda31fb4d305f8c0771d2b6a00f12874094354f3643d80410ca1
  Stored in directory: /home/visionadmin/.cache/pip/wheels/0d/7c/fb/05123381bd60c57ffcdc6fcc1c26e585dedee85b8c1625e2c1
Successfully built paho-mqtt
     |████████████████████████████████| 106 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 16.3 MB/s eta 0:00:01


## Import A Helper Python Class to Deploy Media Graphs
The following code snippet will import a custom Python class to help us deploy media edge graphs. After the import, we will instantiate a graph manager object with IoT Hub, IoT Edge Device, and Graph API version details.

In [27]:
from env_variables import *
from graph_manager import GraphManager

moduleId = "lvaEdge" # Must be same as the name that we assigned to LVA module in the deployment manifest file
operationsApiVersion = "1.0"  # Must be same as the version number in the deployment manifest file

graphManager = GraphManager(iotHubConnString, iotDeviceId, moduleId, operationsApiVersion)

## List Existing Graph Topologies and Instances


In [28]:
# List topologies
response = graphManager.GenericCall("GraphTopologyList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


In [21]:
# List instances
response = graphManager.GenericCall("GraphInstanceList", {})
print(response)

{'additional_properties': {}, 'status': 200, 'payload': {'value': []}}


## Set Topology
The following code snippet will deploy the "template.json" file into the lvaEdge module running on our IoT Edge device.

In [22]:
# You can define a topologyFile or a topologyUrl that you want to deploy into the module. Here we point to our sample Media Graph Topology File.
operationParams = {
                    "topologyFile": "../../../../MediaGraph/topologies/motion-with-httpExtension/topology.json"
                    }

In [23]:
# Set Graph Topology
response = graphManager.GraphTopologySet(operationParams)
print(response)

{'additional_properties': {}, 'status': 201, 'payload': {'systemData': {'createdAt': '2020-07-07T19:20:29.085Z', 'lastModifiedAt': '2020-07-07T19:20:29.085Z'}, 'name': 'EVROnMotionPlusHttpExtension', 'properties': {'description': 'Event-based video recording to Assets based on motion events, and using HTTP Extension to send images to an external inference engine', 'parameters': [{'name': 'rtspUserName', 'type': 'String', 'description': 'rtsp source user name.', 'default': 'dummyUserName'}, {'name': 'rtspPassword', 'type': 'String', 'description': 'rtsp source password.', 'default': 'dummyPassword'}, {'name': 'rtspUrl', 'type': 'String', 'description': 'rtsp Url'}, {'name': 'motionSensitivity', 'type': 'String', 'description': 'motion detection sensitivity', 'default': 'medium'}, {'name': 'httpAIServerAddress', 'type': 'String', 'description': 'AI Server Address', 'default': 'http://yolov3/score'}, {'name': 'hubSinkOutputName', 'type': 'String', 'description': 'hub sink output name', 'd

## Set Topology Instance

The following code snippet will set the values of the parameters mentioned in "template.json" and will create a Topology Instance. If you modify the topology or add new parameters, you should update the parameter list below accordingly.  

>[!IMPORTANT]  
>- Be sure that the value of "topologyName" parameter below and the same parameter in the "template.json" file are the same.  
>- Since we don't have a physical IP camera set for this sample, we use a virtual IP camera simulator. In the previous steps when we deployed the module deployment manifest, we already deployed a simulator module which plays "lots_284.mkv" video file as if played from an IP camera. Thus, in the parameter list below, the parameter "rtspUrl" points to the URL address of this IoT Module (rtsp://rtspsim:554) with the full path of the video file that we want to play. You can replace this parameter's value with full RTSP address of your phyical IP camera.  
>- rtspUsername and rtspPassword are dummy values for our simulator because they do not require authentication. However, if your source stream (i.e., your phyical IP Camera) requires authentication, then put the appropriate values for these parameters.  
>- The value of the "motionSensitivity" parameter can be one of: {low, medium, high}. As the name implies, it sets the sensitivity of the motion detection processor.
>- The value of the "grpcAIServerAddress" parameter points to the address of the inference server. The name of the module that we developed in this sample and deployed in our module deployment manifest was set to "isServer", which was listening to an inference request on "/score" endpoint through port 44001. Here, we set the exact address of the scoring endpoint.
>- The "hubSinkOutputName" parameter sets the IoT Hub channel name which ingests inference results into IoT Hub.

In [24]:
mediaGraphTopologyParameters = {
          "name": "Sample-Graph-1",
          "properties": {
            "topologyName": "EVROnMotionPlusHttpExtension",
            "description": "Sample graph description",
            "parameters": [
              {
                "name": "rtspUserName",
                "value": "username"
              },
              {
                "name": "rtspPassword",
                "value": "password"
              },
              {
                "name": "rtspUrl",
                "value": "rtsp://rtspsim:554/media/lots_284.mkv"
              },
              {
                "name": "motionSensitivity",
                "value": "medium"
              },
              {
                "name": "httpAIServerAddress",
                "value": "http://isEdge:5001/score"
              },
              {
                "name": "hubSinkOutputName",
                "value": "inferences"
              },
              {
                "name": "imageEncoding",
                "value": "jpeg"
              },
              {
                "name": "imageScaleMode",
                "value": "pad"
              }
            ]
          }
        }

After setting the parameters above (as Python dictionary data structure), we can now set an instance of the previously deployed topology on the Edge device with the custom parameters above.

In [25]:
# Set topology instance
response = graphManager.GenericCall("GraphInstanceSet", mediaGraphTopologyParameters)
print(response)

{'additional_properties': {}, 'status': 201, 'payload': {'systemData': {'createdAt': '2020-07-07T19:21:35.760Z', 'lastModifiedAt': '2020-07-07T19:21:35.760Z'}, 'name': 'Sample-Graph-1', 'properties': {'state': 'Inactive', 'description': 'Sample graph description', 'topologyName': 'EVROnMotionPlusHttpExtension', 'parameters': [{'name': 'rtspUserName', 'value': 'username'}, {'name': 'rtspPassword', 'value': 'password'}, {'name': 'rtspUrl', 'value': 'rtsp://rtspsim:554/media/lots_284.mkv'}, {'name': 'motionSensitivity', 'value': 'medium'}, {'name': 'httpAIServerAddress', 'value': 'http://isEdge:5001/score'}, {'name': 'hubSinkOutputName', 'value': 'inferences'}, {'name': 'imageEncoding', 'value': 'jpeg'}, {'name': 'imageScaleMode', 'value': 'pad'}]}}}


## Activate Topology Instance
Next, we activate the Topology Instance that we set in the previous node.

>[!IMPORTANT]  
>Be sure to set the name parameter below to the exact topology instance name that we used in the previous node.

In [26]:
# Activate topology instance
operationParams = {
                    "name": "Sample-Graph-1"
                    }

response = graphManager.GenericCall("GraphInstanceActivate", operationParams)
print(response)

{'additional_properties': {}, 'status': 200, 'payload': None}


## Next Steps

If all the code cells above have successfully finished running, return to the Readme page to continue.   